In [1]:
import pandas as pd
import dask.dataframe as dd

In [2]:
from dask.distributed import Client
client = Client()

In [3]:
df = dd.read_csv("synonymous_gnomAD_ukb_denovo/*part", sep = "\t")

In [5]:
df.head()

,Chrom,Pos,Allele_ref,Allele,gnomADv2_polymorphic,denovo_genome,denovo_exome,mu,mu_quality,Carlson_rate,gnomAD_rate,mean,median,ukb_polymorphic,filter_ukb
0,1,65570,G,A,0,NaN,NaN,0.105,low,NaN,1.009018e-08,NaN,NaN,0,NaN
1,1,65573,G,A,0,NaN,NaN,0.223,low,NaN,1.512832e-08,NaN,NaN,0,NaN
2,1,69039,A,C,0,NaN,NaN,0.013,low,NaN,2.626783e-09,NaN,NaN,0,NaN
3,1,69039,A,G,0,NaN,NaN,0.062,low,NaN,7.966639e-09,NaN,NaN,0,NaN
4,1,69039,A,T,0,NaN,NaN,0.020,low,NaN,3.122595e-09,NaN,NaN,0,NaN


### filter for sites with mutation rate estimates

In [6]:
rate_column_name = ["mu", "Carlson_rate", "gnomAD_rate"]
for i in rate_column_name:
    df = df[df[i].isna() == False]
    df = df[df[i] != 0]

In [7]:
df = df[df["mu_quality"] == "high"]

### output hash table

In [11]:
mu_list = ["mu", "Carlson_rate", "gnomAD_rate"]

polymorphic_list = ["gnomADv2_polymorphic", "ukb_polymorphic", "denovo_genome", "denovo_exome"]

In [12]:
for mu in mu_list:
    for polymorphic in polymorphic_list:
        print(polymorphic)
        df_grouped = pd.DataFrame(df.groupby(mu).agg({polymorphic: ['size', 'sum']}).compute())
#         df_group.rename({0: "total"}, axis = 1, inplace = True)
#         df_group = df_group.reset_index()

        df_grouped.to_csv(polymorphic + "_" + mu +"_binned.tsv", sep = "\t")

gnomADv2_polymorphic
ukb_polymorphic
denovo_genome
denovo_exome
gnomADv2_polymorphic
ukb_polymorphic
denovo_genome
denovo_exome
gnomADv2_polymorphic
ukb_polymorphic
denovo_genome
denovo_exome
